In [3]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [4]:
using GLMNet
using RDatasets
using MLBase
using Plots
using Distances
using Random
using LinearAlgebra
using DataStructures


In [5]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [6]:
iris = dataset("datasets", "iris")
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

42-element Vector{Int64}:
   4
  14
  21
  24
  29
  34
  37
  42
  45
  46
  54
  60
  61
   ⋮
 113
 116
 119
 124
 125
 126
 130
 131
 133
 136
 137
 145

In [7]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))


assign_class (generic function with 1 method)

In [12]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9523809523809523